In [34]:
import dowhy
from dowhy import CausalModel
from IPython.display import Image, display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

df = pd.read_csv("lucas0_train.csv")

def gml_to_string(file):
    gml_str = ''
    with open(file, 'r') as file:
        for line in file:
            gml_str += line.rstrip()
    return gml_str


gml_graph = gml_to_string('lucas_graph.gml')

# With GML string
model=CausalModel(
        data = df,
        treatment='Lung_cancer',
        outcome='Car_Accident',
        graph=gml_graph
        )
identified_estimand = model.identify_effect()
print(identified_estimand)


Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                                           
──────────────(Expectation(Car_Accident|Genetics))
d[Lung_cancer]                                    
Estimand assumption 1, Unconfoundedness: If U→{Lung_cancer} and U→Car_Accident then P(Car_Accident|Lung_cancer,Genetics,U) = P(Car_Accident|Lung_cancer,Genetics)

### Estimand : 2
Estimand name: iv
Estimand expression:
Expectation(Derivative(Car_Accident, [Smoking])*Derivative([Lung_cancer], [Smo
king])**(-1))
Estimand assumption 1, As-if-random: If U→→Car_Accident then ¬(U →→{Smoking})
Estimand assumption 2, Exclusion: If we remove {Smoking}→{Lung_cancer}, then ¬({Smoking}→Car_Accident)

### Estimand : 3
Estimand name: frontdoor
Estimand expression:
Expectation(Derivative(Car_Accident, [Fatigue])*Derivative([Fatigue], [Lung_ca
ncer]))
Estimand assumption 1, Full-mediation: Fatigue intercepts (blocks) all directed paths from Lung_cancer to C

In [35]:
identifier = model.identifier
identifier.identify_backdoor(identifier.treatment_name, identifier.outcome_name)
df

,Smoking,Yellow_Fingers,Anxiety,Peer_Pressure,Genetics,Attention_Disorder,Born_an_Even_Day,Car_Accident,Fatigue,Allergy,Coughing,Lung_cancer
0,0,0,1,0,0,1,0,1,0,1,0,0
1,0,1,0,0,0,0,1,0,1,0,1,1
2,1,1,1,0,1,1,1,1,1,1,1,1
3,0,0,0,1,0,0,1,0,0,0,0,0
4,1,1,1,0,0,1,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0,1,0,0,0,0,1,1,0,1,1
1996,1,1,1,0,0,0,0,0,1,0,1,1
1997,1,1,0,1,0,0,1,0,1,0,0,1
1998,1,1,1,0,1,1,0,1,1,0,1,1


In [36]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                                           
──────────────(Expectation(Car_Accident|Genetics))
d[Lung_cancer]                                    
Estimand assumption 1, Unconfoundedness: If U→{Lung_cancer} and U→Car_Accident then P(Car_Accident|Lung_cancer,Genetics,U) = P(Car_Accident|Lung_cancer,Genetics)

## Realized estimand
b: Car_Accident~Lung_cancer+Genetics+Lung_cancer*Attention_Disorder+Lung_cancer*Allergy
Target units: ate

## Estimate
Mean value: 0.1536794644906877
### Conditional Estimates
__categorical__Attention_Disorder  __categorical__Allergy
(-0.001, 1.0]                      (-0.001, 1.0]             0.153679
dtype: float64
Causal Estimate is 0.1536794644906877


In [30]:
from sklearn.linear_model import LinearRegression

# we fit a linear regression model with v0 and y, and check what the estimated linear effect is
lr = LinearRegression() 

lr.fit(df[["Genetics", "Lung_cancer"]], df["Car_Accident"])
lr.coef_

array([0.14659341, 0.14269749])

In [31]:
lr = LinearRegression() 

lr.fit(df[["Attention_Disorder", "Lung_cancer"]], df["Car_Accident"])
lr.coef_

array([0.28012755, 0.1485292 ])

In [77]:
model=CausalModel(
        data = df.astype(bool),
        treatment='Lung_cancer',
        outcome='Car_Accident',
        graph=gml_graph
        )
identified_estimand = model.identify_effect()

causal_estimate = model.estimate_effect(identified_estimand,
        method_name="iv.instrumental_variable")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))



*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: iv
Estimand expression:
Expectation(Derivative(Car_Accident, [Smoking])*Derivative([Lung_cancer], [Smo
king])**(-1))
Estimand assumption 1, As-if-random: If U→→Car_Accident then ¬(U →→{Smoking})
Estimand assumption 2, Exclusion: If we remove {Smoking}→{Lung_cancer}, then ¬({Smoking}→Car_Accident)

## Realized estimand
Realized estimand: Wald Estimator
Realized estimand type: nonparametric-ate
Estimand expression:
                                                                              
Expectation(Derivative(Car_Accident, Smoking))⋅Expectation(Derivative(Lung_can

              -1
cer, Smoking))  
Estimand assumption 1, As-if-random: If U→→Car_Accident then ¬(U →→{Smoking})
Estimand assumption 2, Exclusion: If we remove {Smoking}→{Lung_cancer}, then ¬({Smoking}→Car_Accident)
Estimand assumption 3, treatment_effect_homogeneity: Each unit's treatment ['Lung_cancer'] is a

In [75]:
df = df.astype(bool)

IV = "Smoking"
T = "Lung_cancer"
Y = "Car_Accident"
smoking = df[df[IV]]
not_smoking = df[~df[IV]]

(smoking[Y].mean() - not_smoking[Y].mean()) / \
(smoking[T].mean() - not_smoking[T].mean())

/home/bart2/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_png function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/home/bart2/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_rgba function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/home/bart2/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_mask function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/home/bart2/.pyenv/versions/3.9.5/lib/python3.9/site-packages/

In [73]:
X = "Genetics"
T = "Lung_cancer"
Y = "Car_Accident"

p00 = (~df[~df[X]][T]).mean()
p01 = (~df[df[X]][T]).mean()
p10 = df[~df[X]][T].mean()
p11 = df[df[X]][T].mean()
n = len(df.index)

y_0 = (df[~df[T] & df[X]].sum() / p01 + df[~df[T] & ~df[X]].sum() / p00) / n
y_1 = (df[df[T] & df[X]].sum() / p11 + df[df[T] & ~df[X]].sum() / p10) / n

(y_1 - y_0)[Y]




/home/bart2/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_png function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/home/bart2/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_rgba function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/home/bart2/.pyenv/versions/3.9.5/lib/python3.9/site-packages/IPython/lib/latextools.py:126: MatplotlibDeprecationWarning: 
The to_mask function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use mathtext.math_to_image instead.
  mt.to_png(f, s, fontsize=12, dpi=dpi, color=color)
/home/bart2/.pyenv/versions/3.9.5/lib/python3.9/site-packages/